## <center> Data engineer Home case


SUMMARY

Rosstat periodically provides different data about agriculture market.

The task is to automate the process of gathering and storing sample data (Harvested area) into database (using Python) and make some analytics based on downloaded data (using SQL)



TASKS:

1. Python
Write the python script (Jupyter Notebook preferable), that:

* Parses the data from https://www.fedstat.ru/indicator/31328


for every:
year [ 2016 – last available year], product [пшеница яровая, пшеница озимая, кукуруза на зерно], region	

In [36]:
import requests
from bs4 import BeautifulSoup

import os, os.path

import re

In [37]:
# Запрос к указанной странице и преобразование полученных данных к читабельному виду

url = 'https://www.fedstat.ru/indicator/31328'
response = requests.get(url)

print(response.status_code)

page = BeautifulSoup(response.text, 'html.parser')


200


Результат предыдущего запроса содержит информацию:

window['grid'] = new FGrid({
                        id: 31328,
                        block: $('#grid'),
                        debug: true,
                        title: '\u041F\u043E\u0441\u0435\u0432\u043D\u044B\u0435 \u043F\u043B\u043E\u0449\u0430\u0434\u0438 \u0441\u0435\u043B\u044C\u0441\u043A\u043E\u0445\u043E\u0437\u044F\u0439\u0441\u0442\u0432\u0435\u043D\u043D\u044B\u0445 \u043A\u0443\u043B\u044C\u0442\u0443\u0440',
                        unit: '\u0442\u044B\u0441\u044F\u0447\u0430 \u0433\u0435\u043A\u0442\u0430\u0440\u043E\u0432',
                        filters: {
                            0: {
                                title: '\u041F\u043E\u043A\u0430\u0437\u0430\u0442\u0435\u043B\u044C',
                                all: true,
                                values: {
                                    31328: {
                                        title: '\u041F\u043E\u0441\u0435\u0432\u043D\u044B\u0435 \u043F\u043B\u043E\u0449\u0430\u0434\u0438 \u0441\u0435\u043B\u044C\u0441\u043A\u043E\u0445\u043E\u0437\u044F\u0439\u0441\u0442\u0432\u0435\u043D\u043D\u044B\u0445 \u043A\u0443\u043B\u044C\u0442\u0443\u0440',
                                        order: 0,
                                        checked: true
                                    }
                                },
                                indicator: true
                            },
                            3: {
                                title: '\u0413\u043E\u0434',
                                all: false,
                                values: {
                                    1990: {
                                        title: '1990',
                                        order: 0,
                                        checked: false
...

Из этого можно извлечь коды полей, с которыми в дальнейшем предстоит работать, чтобы извлечь конкетные цифры. Для этого пришлось сопоставить структуру New FGrid с информацией на сайте.

Периоды: поле имеет код 3, значениями являются годы, нас интересуют 2016-2021

Единица измерения: код 0, значение 31328 - тысяча гектаров (совпадает с кодом индикатора)

Хозяйства: код поля 58423, нас интересует значение "все", его код 1750642

Сельхоз культуры: поле 30611, значения 14 - кукуруза на зерно, 28 - пшеница озимая и 30 - пшеница яровая

Регионы: поле 57831. Нас интересуют не все значения, так как в справочнике есть значения

 Российская Федерация или Южный Федеральный округ, содержащие итоги. Но возможно проще будет выбрать все, и исключить лишние на следующем шаге.

In [38]:
# Здесь необходимо составить структуру следующего запроса к сайту ЕМИСС на основе информации выше,
# чтобы получить конкретные данные

# Мне было непонятно, к какой странице обращаться, чтобы получить собственно данные,
# и каким образом перечислить в запросе указанные выше поля.

# Поэтому использовала чужое решение в открытом доступе, созданное для предыдущей версии сайта ЕМИСС
# https://github.com/infoculture/hubofdata/blob/master/scripts/catalogs/fedstatru/emiss_extract.py

url_data = 'https://fedstat.ru/indicator/data.do'

the_word='new FGrid'
data=str(page.findAll('script',text=lambda t: t and the_word in t)[0])

found = re.search('title: \'(.+)', data).group()
title=found.split(': ')[1]

data_post=[('lineObjectIds','0')]
try:
    found = re.search('left_columns: \[\n(.+)', data).group(1)
except AttributeError:
    found = '' 

for i in found.strip().split(', '):
    data_post.append(('lineObjectIds',i))

try:
    found = re.search('top_columns: \[\n(.+)', data).group(1)
except AttributeError:
    found = '' 


for i in found.strip().split(', '):
    data_post.append(('columnObjectIds',i))

i=0
prefix=''
suffix=''
num=data.find('filters')

for letter in data[num:].splitlines():
    if '{' in letter:
        i+=1
    if '}' in letter:
        i-=1
    if re.findall('\d+: {' ,letter):
        if i<=2:
            prefix=letter.strip().split(':')[0]
        if i>2:
            suffix=letter.strip().split(':')[0]
    data_post.append(('selectedFilterIds',prefix+'_'+suffix))
    
data_post.append(('id', '31328'))
params = (('format', 'excel'),)


resp = requests.post(url_data, params=params, data=data_post)


In [39]:
# Следюущий шаг - переформатировать данные для запроса data_post, чтобы получить файл нужного нам формата
 
data_post[500:]

[('selectedFilterIds', '57831_1688540'),
 ('selectedFilterIds', '57831_1688541'),
 ('selectedFilterIds', '57831_1688541'),
 ('selectedFilterIds', '57831_1688541'),
 ('selectedFilterIds', '57831_1688541'),
 ('selectedFilterIds', '57831_1688541'),
 ('selectedFilterIds', '57831_1688542'),
 ('selectedFilterIds', '57831_1688542'),
 ('selectedFilterIds', '57831_1688542'),
 ('selectedFilterIds', '57831_1688542'),
 ('selectedFilterIds', '57831_1688542'),
 ('selectedFilterIds', '57831_1688543'),
 ('selectedFilterIds', '57831_1688543'),
 ('selectedFilterIds', '57831_1688543'),
 ('selectedFilterIds', '57831_1688543'),
 ('selectedFilterIds', '57831_1688543'),
 ('selectedFilterIds', '57831_1688544'),
 ('selectedFilterIds', '57831_1688544'),
 ('selectedFilterIds', '57831_1688544'),
 ('selectedFilterIds', '57831_1688544'),
 ('selectedFilterIds', '57831_1688544'),
 ('selectedFilterIds', '57831_1688545'),
 ('selectedFilterIds', '57831_1688545'),
 ('selectedFilterIds', '57831_1688545'),
 ('selectedFilte

In [40]:
# Сохранение данных в Excel

f = open('data_31328.xlsx', 'wb')
f.write(resp.content)
f.close()

* Inserts these data into database table (Postgres preferable) with the structure:

region_name, begin_date, end_date, product_type, volume

In [41]:
# Здесь будет блок кода по переносу данных из Excel в базу Postgres